# Creating a feature matrix from a DB Query (Opto Sessions)

In [62]:
import sys
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit/data_preprocessing_code/opto_preprocessing')
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing_opto as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [63]:
#load in data base
db = pd.read_csv('/Users/Sabatini Lab/GitHub/mouse_bandit/mike_session_record.csv',index_col=0)

### Query all 90-10 sessions where performance > 0.5 block structure was 50 and had laser trials @ 10% of trials

In [64]:
r = db[((db['Left Reward Prob'] == 0.7) |  (db['Right Reward Prob'] == 0.7))].copy()
r = r[r['p(high Port)'] > 0.5].copy()
r = r[r['Block Range Min'] == 50].copy()
r = r[r['Laser Stim Prob'] == 0.1].copy()

session_names = r['Session ID'].values 

In [65]:
    r.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Laser Stim Prob,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
1196,mike,08252017_SOM41,SOM41,2017-08-25 00:00:00,2,0.1,0.7,0.3,50,50,568,6,332,0.69,2,1.0,40,40
1197,mike,08252017_SOM44,SOM44,2017-08-25 00:00:00,2,0.1,0.3,0.7,50,50,442,5,254,0.64,2,1.0,40,40
1198,mike,08252017_SOM45,SOM45,2017-08-25 00:00:00,2,0.1,0.7,0.3,50,50,695,8,401,0.68,2,1.0,40,40
1203,mike,08282017_SOM44,SOM44,2017-08-28 00:00:00,2,0.1,0.3,0.7,50,50,566,6,330,0.77,2,1.0,40,40
1204,mike,08282017_SOM45,SOM45,2017-08-28 00:00:00,2,0.1,0.3,0.7,50,50,683,8,406,0.73,2,1.0,40,40
1215,mike,08302017_SOM44,SOM44,2017-08-30 00:00:00,2,0.1,0.3,0.7,50,50,559,6,307,0.64,2,1.0,40,40


In [66]:
r.shape

(6, 18)

## load in csv files (from running exportTrials.m) added in 'Laser Given' column in trials for new data format following opto changes

In [67]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given','Laser Given']

root_dir = '/Users/Sabatini Lab/GitHub/mouse_bandit/data/mike_data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

## convert into 1 feature matrix

In [68]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [69]:
master_matrix.shape

(3453, 64)

In [70]:
master_matrix['Higher p port'].mean()

0.70026064291920065

In [71]:
master_matrix['Laser'].mean()

0.10599478714161599

## Stephenson et al acquired ~6500 trials per animal

In [72]:
master_matrix['Session ID'].unique()

array(['08252017_SOM41', '08252017_SOM44', '08252017_SOM45',
       '08282017_SOM44', '08282017_SOM45', '08302017_SOM44'], dtype=object)

In [73]:
pd.set_option('display.max_columns', None)

In [47]:
master_matrix.tail(10)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,10_trialDuration,10_laser,9_Port,9_Reward,9_ITI,9_trialDuration,9_laser,8_Port,8_Reward,8_ITI,8_trialDuration,8_laser,7_Port,7_Reward,7_ITI,7_trialDuration,7_laser,6_Port,6_Reward,6_ITI,6_trialDuration,6_laser,5_Port,5_Reward,5_ITI,5_trialDuration,5_laser,4_Port,4_Reward,4_ITI,4_trialDuration,4_laser,3_Port,3_Reward,3_ITI,3_trialDuration,3_laser,2_Port,2_Reward,2_ITI,2_trialDuration,2_laser,1_Port,1_Reward,1_ITI,1_trialDuration,1_laser,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward,Laser
663,SOM45,08282017_SOM45,674.0,7.0,1.0,81.0,-5.0,0.613,1.0,1.0,1.576,0.444,0.0,1.0,1.0,1.319,0.553,0.0,1.0,1.0,1.471,0.470,0.0,1.0,1.0,1.523,0.327,0.0,1.0,1.0,1.605,0.585,0.0,1.0,0.0,1.163,0.346,0.0,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.593,1.0,0.0,0.0,0.0,1.0
664,SOM45,08282017_SOM45,675.0,8.0,1.0,82.0,-6.0,0.338,1.0,1.0,1.319,0.553,0.0,1.0,1.0,1.471,0.470,0.0,1.0,1.0,1.523,0.327,0.0,1.0,1.0,1.605,0.585,0.0,1.0,0.0,1.163,0.346,0.0,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.199,1.0,0.0,0.0,0.0,0.0
665,SOM45,08282017_SOM45,676.0,9.0,1.0,83.0,-7.0,0.537,1.0,1.0,1.471,0.470,0.0,1.0,1.0,1.523,0.327,0.0,1.0,1.0,1.605,0.585,0.0,1.0,0.0,1.163,0.346,0.0,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.187,1.0,0.0,0.0,0.0,0.0
666,SOM45,08282017_SOM45,677.0,10.0,1.0,84.0,-8.0,0.560,1.0,1.0,1.523,0.327,0.0,1.0,1.0,1.605,0.585,0.0,1.0,0.0,1.163,0.346,0.0,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,1.174,0.0,1.0,1.0,1.0,0.0
667,SOM45,08282017_SOM45,678.0,11.0,2.0,1.0,1.0,0.370,1.0,1.0,1.605,0.585,0.0,1.0,0.0,1.163,0.346,0.0,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,0.0,1.0,1.174,0.560,0.0,1.282,0.0,0.0,1.0,1.0,0.0
668,SOM45,08282017_SOM45,679.0,12.0,3.0,2.0,2.0,0.474,1.0,0.0,1.163,0.346,0.0,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,0.0,1.0,1.174,0.560,0.0,0.0,1.0,1.282,0.370,0.0,1.518,0.0,0.0,1.0,0.0,0.0
669,SOM45,08282017_SOM45,680.0,13.0,3.0,3.0,-1.0,0.360,1.0,0.0,1.239,0.392,0.0,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,0.0,1.0,1.174,0.560,0.0,0.0,1.0,1.282,0.370,0.0,0.0,0.0,1.518,0.474,0.0,1.046,0.0,0.0,1.0,1.0,0.0
670,SOM45,08282017_SOM45,681.0,14.0,4.0,4.0,1.0,0.557,1.0,0.0,1.278,0.548,0.0,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,0.0,1.0,1.174,0.560,0.0,0.0,1.0,1.282,0.370,0.0,0.0,0.0,1.518,0.474,0.0,0.0,1.0,1.046,0.360,0.0,2.457,1.0,1.0,0.0,0.0,1.0
671,SOM45,08282017_SOM45,682.0,15.0,4.0,1.0,-1.0,0.552,1.0,0.0,1.243,0.526,1.0,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,0.0,1.0,1.174,0.560,0.0,0.0,1.0,1.282,0.370,0.0,0.0,0.0,1.518,0.474,0.0,0.0,1.0,1.046,0.360,0.0,1.0,0.0,2.457,0.557,1.0,1.281,1.0,0.0,0.0,0.0,0.0
672,SOM45,08282017_SOM45,683.0,16.0,4.0,2.0,-2.0,0.379,1.0,0.0,3.178,0.481,0.0,1.0,0.0,1.593,0.613,1.0,1.0,0.0,1.199,0.338,0.0,1.0,0.0,1.187,0.537,0.0,0.0,1.0,1.174,0.560,0.0,0.0,1.0,1.282,0.370,0.0,0.0,0.0,1.518,0.474,0.0,0.0,1.0,1.046,0.360,0.0,1.0,0.0,2.457,0.557,1.0,1.0,0.0,1.281,0.552,0.0,1.125,0.0,1.0,1.0,1.0,0.0


## Save combined feature matrix  

In [74]:
master_matrix.to_csv(os.path.join(root_dir,'0.1opto_trials_7030_high.csv'))

In [31]:
master_matrix.to_csv(os.path.join(root_dir,'0.1opto_trials_9010_high.csv'))